Inga Ulusoy, Computational modelling in python, SoSe2020 

# Interpolation of functions

Imagine you know only values of a function at specific points, but in fact you need values at arbitrary points in between. This is achieved through interpolation - the function is modeled by a plausible functional form. 

The interpolation process has two stages:
1. Fit the interpolating function to reproduce the existing values.
2. Evaluate the interpolating function at the target point.

## 1. Read in the data

In [ ]:
from math import *
from numpy import *
import matplotlib.pyplot as plt

In [ ]:
nofuncs=4
nopoints=4
myval=zeros((nofuncs,nopoints))

with open('data_problem3.dat') as f:
    for line in f:
        myline=line.split()
        j=int(myline[0])-1
        i=int(myline[1])-1
        myval[i,j]=float(myline[2])
        
print(myval)

In [ ]:
mf=16
fig, ax = plt.subplots(figsize=(8,5))
for i in range(nofuncs):
    ax.plot(myval[i],marker='<',label='linear, f{}'.format(i+1))

ax.set_xlabel('x point',fontsize=18)
ax.set_ylabel('y value',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
legend = ax.legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
#plt.savefig('problem3_linear.pdf',dpi=300,bbox_inches='tight')
plt.show()

## 2. Linear interpolation

In linear interpolation, two neighboring points $x_j$ and $x_{j+1}$ are connected by a straight line, and the linear function between these two points provides the mapping for arbitrary points in the interval $[x_j,x_{j+1}]$.
\begin{align}
f(x)=A f(x_j)+B f(x_{j+1})
\end{align}
For $n$ points that are being interpolated, the overall function is $n-1$ piecewise linear, as shown in the plot above - each of the given points is connected to the next point by a straight line. Linear interpolation is a special case of polynomial interpolation.

The coefficients are obtained as:
\begin{align}
A & = \frac{x_{j+1}-x}{x_{j+1}-x_j} \\
B & = 1 - A = \frac{x-x_j}{x_{j+1}-x_j}
\end{align}
The coefficients $A$ and $B$ provide the weights of each data point, with the closer data point having a greater weight in the interpolation.

We will now perform a linear interpolation for the four functions above, onto a finer mesh of 16 evenly spaced points, from the given four data points. The new points are computed according to
\begin{align}
f(x) = f(x_j) + (x-x_j) \frac{y_{j+1}-y_j}{x_{j+1}-x_j}
\end{align}

*Note that there are many different ways to write and arrange these routines, we will start with a more simple and sequential syntax.*

In [ ]:
nint=12
xval=zeros((nofuncs,nint))
linval=zeros((nofuncs,nint))
#spacing between the new x values
#we have nopoints-1 intervals that are equally divided into nint pieces
dx = (nopoints-1)/nint
print('The spacing between the interpolated points is',dx)
#determine the x values for all four functions
for j in range(nint):
    xval[:,j]=j*dx
#now compute the y values
for i in range(nofuncs):
    for j in range(nint):
        #find out which interval we are in - in this case, we simply need the 
        #integer value of the grid point x
        myj=int(xval[i,j])
        myjplusone=myj+1
        yj=myval[i,myj]
        yjplusone=myval[i,myjplusone]
        #now we can compute the new y value for the given x
        linval[i,j] = yj + (xval[i,j]-myj) * (yjplusone-yj)/(myjplusone-myj)

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

fig, ax = plt.subplots(figsize=(8,5))
for i in range(nofuncs):
    ax.plot(xval[i],linval[i],marker='x',color=colors[i],label='linear, f{}'.format(i+1))
    ax.plot(myval[i],marker='<',color=colors[i])

ax.set_xlabel('x point',fontsize=18)
ax.set_ylabel('y value',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
legend = ax.legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
#plt.savefig('problem3_linear.pdf',dpi=300,bbox_inches='tight')
plt.show()

This can all be done much easier using the numpy library:

In [ ]:
linvals2=zeros((nofuncs,nint))
myxold=arange(0,nopoints)
for i in range(nofuncs):
    linvals2[i] = interp(xval[i], myxold, myval[i])

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for i in range(nofuncs):
    ax.plot(xval[i],linvals2[i],marker='x',color=colors[i],label='linear, f{}'.format(i+1))
    ax.plot(myval[i],marker='<',color=colors[i])

ax.set_xlabel('x point',fontsize=18)
ax.set_ylabel('y value',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
legend = ax.legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
#plt.savefig('problem3_linear.pdf',dpi=300,bbox_inches='tight')
plt.show()

__Note how easy this is using the library! Generally, we will employ available libraries from now on and not implement the numerical methods from scratch!!__

A huge disadvantage of linear interpolation is that the derivatives are not continuous:

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for i in range(nofuncs):
    ax.plot(xval[i,0:(nint-1)],[linval[i,j+1]-linval[i,j] for j in range(nint-1)],marker='x',label='deriv, f{}'.format(i+1))
    #ax.plot(myval[i],marker='<')

ax.set_xlabel('x point',fontsize=18)
ax.set_ylabel('y value',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.title('The first derivative is not smooth over the range of values.',fontsize=mf)
legend = ax.legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
#plt.savefig('problem3_linear.pdf',dpi=300,bbox_inches='tight')
plt.show()

For continuous first and second derivatives, we need to resort to a different numerical approach.

## Cubic spline interpolation

With a cubic spline interpolation, smooth first derivatives and continuous second derivatives are obtained for each interpolated point.


Another advantage is that, contrary to standard polynomial interpolation where an $(n-1)$-degree polynomial is fitted to $n$ data points, overfitting is avoided. (Overfitting occurs for example in polynomial interpolation when simple data is represented with an excessively complicated form, and manifests itself in an oscillatory behaviour of the fit.) 

This is achieved by fitting to a functional form of
\begin{align}
f(x) = A f(x_j) +  B f (x_{j+1}) + C f''(x_j) + D f'' (x_{j+1})
\end{align}
A natural cubic spline has zero value for $f''(x_0)$ and $f''(x_n)$ and is the most typical case.

In this approach, each segment between two neighboring data points is represented by its own polynomial $P(x)$, resulting in a piecewise polynomial function $S(x)$ of $n-1$ polynomials ("spline"):
\begin{align}
S(x) =
\left\{
\begin{array}{cc}
P_1(x) & x_0 \le x \le x_1 \\
P_2(x) & x_1 \le x \le x_2\\
\vdots & \\
P_n(x)  & x_{n-1} \le x \le x_n \\
\end{array}
\right.
\end{align}
To obtain a continuous and smooth function $S(x)$ and derivatives, each consecutive polynomial $P_i(x)$ and $P_{i+1}(x)$ must have the same functional value and derivatives at the joining points $x_i$. In cubic spline interpolation, the polynomials $P_i(x)$ are cubic polynomials:
\begin{align}
P_i(x)=a_i x^3 + b_i x^2 + c_i x + d_i
\end{align}
Using the boundary conditions for a natural spline, the equations that are obtained for the coefficients are tridiagonal linear, where each polynomial is only coupled to its nearest neighbors:

\begin{align}
\left[
\begin{array}{7c}
2 & 1 &   &   &   &   &      \\
1 & 4 & 1 &   &   &   &      \\
  & 1 & 4 & 1 &   &   &      \\
  &   & 1 & 4 & 1 &   &      \\
  &   &   & \ddots & \ddots & \ddots  &  \\
  &   &   &       & 1 & 4 & 1  \\
  &   &   &       &   & 1 & 2  \\
\end{array}
\right]
\left[
\begin{array}{c}
P'_1 \\
P'_2 \\
P'_3 \\
P'_4 \\
\vdots \\
P'_{n-1} \\
P'_n \\
\end{array}
\right]
=
\left[
\begin{array}{c}
3(x_1-x_0) \\
3(x_2-x_1) \\
3(x_3-x_2) \\
3(x_4-x_3) \\
\vdots \\
3(x_{n-1}-x_{n-2}) \\
3(x_n-x_{n-1}) \\
\end{array}
\right]
\end{align}

A cubic spline interpolation can easily be performed using scipy.

In [ ]:
from scipy.interpolate import CubicSpline
xval2=linspace(0,nopoints-1,16)
csval=zeros((nofuncs),dtype=ndarray)
for i in range(nofuncs):
    csval[i] = CubicSpline(myxold, myval[i])

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for i in range(nofuncs):
    ax.plot(xval2,csval[i](xval2),marker='x',color=colors[i],label='cubic spline, f{}'.format(i+1))
    ax.plot(myval[i],marker='<',color=colors[i],linestyle='')

ax.set_xlabel('x point',fontsize=18)
ax.set_ylabel('y value',fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
legend = ax.legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
#plt.savefig('problem3_cubicspline.pdf',dpi=300,bbox_inches='tight')
plt.show()

We can see that the first and second derivatives are continuous.

In [ ]:
noderivs=2
tt = ['First','Second']
fig, ax = plt.subplots(noderivs,figsize=(8,4*noderivs))
for j in range(noderivs):
    for i in range(nofuncs):
        ax[j].plot(xval2,csval[i](xval2,j+1),marker='x',color=colors[i],label='f{}'.format(i+1))
    ax[j].set_title('{} derivatives'.format(tt[j]),fontsize=mf)
    ax[j].set_xlabel('x point',fontsize=18)
    ax[j].set_ylabel('y value',fontsize=18)
    ax[j].tick_params(labelsize=mf)
    legend = ax[j].legend(loc='lower right', shadow=False,fontsize=mf,borderpad = 0.1, labelspacing = 0, handlelength = 0.8)
plt.tight_layout()
plt.show()

# Problem 3: Best function

The file data_problem3_task1.dat contains the time-dependent dipole moment of HeH$^+$ during irradiation with a laser pulse of 53 fs length. The left column is the time value and the right column contains the dipole moment value in atomic units.

Use the data in data_problem3_task1.dat to perform a linear and a cubic spline interpolation. Plot your results.

In a second experiment, the time-dependent dipole moment of HeH$^+$ was recorded on a much finer time grid. The real data is contained in data_problem3_task2.dat. Compare your fit to the real data - what do you notice?

__Optional additional task (no requirement)__

Repeat the above steps for the data in file data_problem3_task3.dat; this file contains the photodissociation probability of NaI over time. Compare to the real data in data_problem3_task4.dat.

Upload your notebook to moodle.